In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import json
import os
import pickle as pkl
import torch
import openai
from dotenv import load_dotenv
load_dotenv()
from collections import defaultdict

### Khởi tạo model

In [3]:
# with open("knowledge_graph/KG_data/FB15k-237-betae/id2ent.pkl", "rb") as f:
#     id2ent = pkl.load(f)
# with open("knowledge_graph/KG_data/FB15k-237-betae/id2rel.pkl", "rb") as f:
#     id2rel = pkl.load(f)
# with open("knowledge_graph/KG_data/FB15k-237-betae/FB15k_mid2name.txt", "r") as f:
#     ent2name = {}
#     for line in f:
#         mid, name = line.strip().split("\t")
#         ent2name[mid] = name

with open("knowledge_graph/queries/train_2c_id.pkl", "rb") as f:
    query_2_hop = pkl.load(f)
with open("knowledge_graph/queries/train_1c_id.pkl", "rb") as f:
    query_1_hop = pkl.load(f)
with open("knowledge_graph/queries/train_3c_id.pkl", "rb") as f:
    query_3_hop = pkl.load(f)
# with open("knowledge_graph/KG_data/FB15k-237-betae/train-answers.pkl", "rb") as f:
#     answer_train = pkl.load(f)
# with open("knowledge_graph/KG_data/FB15k-237-betae/valid-easy-answers.pkl", "rb") as f:
#     answer_valid_easy = pkl.load(f)
# with open("knowledge_graph/KG_data/FB15k-237-betae/test-hard-answers.pkl", "rb") as f:
#     answer_test_hard = pkl.load(f)
# with open("knowledge_graph/KG_data/FB15k-237-betae/test-easy-answers.pkl", "rb") as f:
#     answer_test_easy = pkl.load(f)

### Tạo embedding cho các relations

Nhận xét:  
+ Direct và inverse relation thường có độ khớp gần như nhau.  

Original query:

In [26]:
query = "Which awards has David Copperfield received for his performances in films?"

#### Query split:  
Do query thể có nhiều phần và cần phải qua nhiều hop mới đạt được kết quả. Do đó, ta có thể đưa query về thành các sub-queries và xử lí từng đoạn.  
+ Splitting queries: Ta có thể đưa LLM làm (PoG có bước này)
+ Compose results: 
    + Sử dụng mean
    + Sử dụng reciprocal rank
    + 

### Inspect queries

In [27]:
query_2_hop[0]

{'query_type': ('e', ('r', 'r')),
 'raw_query': (4582, (133, 17)),
 'named_query': ('Franklin',
  ('-/people/person/places_lived./people/place_lived/location',
   '-/award/award_nominee/award_nominations./award/award_nomination/award_nominee')),
 'transformed_query': ['Who are the award nominees that lived in places associated with Franklin?',
  'Which award nominees have resided in locations tied to Franklin?',
  'Can you tell me the names of award nominees who lived in locations related to Franklin?'],
 'answers_id': [2403, 1221, 137, 1589, 4093],
 'answers': ['Vince_Gill',
  'Albert_Lee',
  'Keith_Urban',
  'Carrie_Underwood',
  'John_Travolta']}

### Test pipeline extract subgraph

In [5]:
from expand_subgraph import ExpandSubgraph
from utils import  extract_numbers, extract_strings,\
                    extract_notations

import argparse
from load_data import DataLoader
import numpy as np

In [6]:
class Config:
    data_path = 'knowledge_graph/KG_data/FB15k-237-betae'
    seed = 1234
    k = 6 # beams
    depth = 8 # max depth of subgraph
    gpu = 0
    fact_ratio = 0.75
    val_num = -1 # how many triples are used as the validate set
    epoch = 200
    layer = 6
    batchsize = 16
    cpu = 1
    weight = ''
    add_manual_edges = False
    remove_1hop_edges = True
    only_eval = False
    not_shuffle_train = False
    device = "cuda:0"

args = Config()

loader = DataLoader(args, mode='train')
# val_loader = DataLoader(args, mode='valid')
# test_loader = DataLoader(args, mode='test')

train_graph = loader.train_graph
train_graph_homo = list(set([(h,t) for (h,r,t) in train_graph]))
# test_data = np.concatenate([np.array(test_data, dtype=np.int32), loader.idd_data], 0, dtype=np.int32)
# train_graph = np.array(train_graph, dtype=np.int32)
args.n_ent = loader.n_ent
args.n_rel = loader.n_rel


==> removing 1-hop links...
==> done


In [8]:
all_subgraph = defaultdict(list)
def print_statistics(stats, label):
    print(f"{label} Statistics:")
    for key, value in stats.items():
        if key in ["mean", "std_dev"]:
            print(f"  {key}: {value:.4f}" if isinstance(value, float) else f"  {key}: {value}")
def calculate_overall_score(query_set, train_sampler, method="default", label=None):
    overall_score = 0
    for query in query_set:
        if method == "default":
            topk_node, _, subgraph = train_sampler.sampleSubgraph(query)
        elif method == "bfs":
            topk_node, _, subgraph = train_sampler.sampleSubgraphBFS(query)
        s = 0
        all_subgraph[label].append({
            "query": query,
            "subgraph": subgraph
        })
        answers = set(query["answers_id"])
        topk_node_set = set(topk_node)
        precision = len(answers & topk_node_set) / len(answers) if len(answers) > 0 else 0
        overall_score += precision
    overall_score /= len(query_set)
    return overall_score

#### P2.s1 approach: Measuring performance

In [41]:
train_sampler = ExpandSubgraph(args.n_ent, args.n_rel,
                               train_graph_homo,train_graph,
                               args=args)

query_1_hop_stat = []
query_2_hop_stat = []
query_3_hop_stat = []

for _ in range(2):
    loader.shuffle_train()
    train_sampler.updateEdges(loader.train_graph)
    random.shuffle(query_1_hop)
    random.shuffle(query_2_hop)
    random.shuffle(query_3_hop)
    query_1_hop_stat.append(calculate_overall_score(query_1_hop[:30], train_sampler, label = "p2s2"))
    query_2_hop_stat.append(calculate_overall_score(query_2_hop[:30], train_sampler, label = "p2s2"))
    query_3_hop_stat.append(calculate_overall_score(query_3_hop[:30], train_sampler, label = "p2s2"))
    print(f"epoch {_}")

epoch 0
epoch 1


In [42]:
from utils import calculate_statistics


stats_1_hop = calculate_statistics(query_1_hop_stat)
print_statistics(stats_1_hop, "1-Hop Query")
stats_2_hop = calculate_statistics(query_2_hop_stat)
print_statistics(stats_2_hop, "2-Hop Query")
stats_3_hop = calculate_statistics(query_3_hop_stat)
print_statistics(stats_3_hop, "3-Hop Query")

1-Hop Query Statistics:
  mean: 0.7939
  std_dev: 0.0058
2-Hop Query Statistics:
  mean: 0.3045
  std_dev: 0.0031
3-Hop Query Statistics:
  mean: 0.1924
  std_dev: 0.0049


#### Vanila BFS

In [ ]:
train_sampler = ExpandSubgraph(args.n_ent, args.n_rel,
                               train_graph_homo,train_graph,
                               args=args)

query_1_hop_stat_bfs = []
query_2_hop_stat_bfs = []
query_3_hop_stat_bfs = []

for _ in range(2):
    loader.shuffle_train()
    train_sampler.updateEdges(loader.train_graph)
    random.shuffle(query_1_hop)
    random.shuffle(query_2_hop)
    random.shuffle(query_3_hop)
    query_1_hop_stat_bfs.append(calculate_overall_score(query_1_hop[:50], train_sampler, "bfs", label="bfs"))
    query_2_hop_stat_bfs.append(calculate_overall_score(query_2_hop[:50], train_sampler, "bfs", label="bfs"))
    query_3_hop_stat_bfs.append(calculate_overall_score(query_3_hop[:50], train_sampler, "bfs", label="bfs"))
    print(f"epoch {_}")

KeyboardInterrupt: 

In [ ]:
stats_1_hop_bfs = calculate_statistics(query_1_hop_stat_bfs)
print_statistics(stats_1_hop_bfs, "1-Hop Query")
stats_2_hop_bfs = calculate_statistics(query_2_hop_stat_bfs)
print_statistics(stats_2_hop_bfs, "2-Hop Query")
stats_3_hop_bfs = calculate_statistics(query_3_hop_stat_bfs)
print_statistics(stats_3_hop_bfs, "3-Hop Query")

1-Hop Query Statistics:
  mean: 0.7970
  std_dev: 0.0118
1-Hop Query Statistics:
  mean: 0.7970
  std_dev: 0.0118
2-Hop Query Statistics:
  mean: 0.4025
  std_dev: 0.0118
3-Hop Query Statistics:
  mean: 0.4431
  std_dev: 0.0123


### Sub-objective approach

In [43]:
train_sampler = ExpandSubgraph(args.n_ent, args.n_rel,
                               train_graph_homo,train_graph,
                               args=args, use_sub_objectives_a=True)

query_1_hop_stat_a = []
query_2_hop_stat_a = []
query_3_hop_stat_a = []

for _ in range(2):
    loader.shuffle_train()
    train_sampler.updateEdges(loader.train_graph)
    random.shuffle(query_1_hop)
    random.shuffle(query_2_hop)
    random.shuffle(query_3_hop)
    query_1_hop_stat_a.append(calculate_overall_score(query_1_hop[:50], train_sampler, label = "subojective"))
    query_2_hop_stat_a.append(calculate_overall_score(query_2_hop[:50], train_sampler, label = "subojective"))
    query_3_hop_stat_a.append(calculate_overall_score(query_3_hop[:50], train_sampler, label = "subojective"))
    print(f"epoch {_}")

openai error, retry: 1 validation error for SubobjectiveOutput
  Invalid JSON: EOF while parsing a string at line 1 column 169 [type=json_invalid, input_value='{"res":["Identify the aw... the awards he has been', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid
epoch 0
epoch 1


In [44]:
from utils import calculate_statistics


stats_1_hop_a = calculate_statistics(query_1_hop_stat_a)
print_statistics(stats_1_hop_a, "1-Hop Query")
stats_2_hop_a = calculate_statistics(query_2_hop_stat_a)
print_statistics(stats_2_hop_a, "2-Hop Query")
stats_3_hop_a = calculate_statistics(query_3_hop_stat_a)
print_statistics(stats_3_hop_a, "3-Hop Query")

1-Hop Query Statistics:
  mean: 0.2864
  std_dev: 0.0002
2-Hop Query Statistics:
  mean: 0.0949
  std_dev: 0.0367
3-Hop Query Statistics:
  mean: 0.0416
  std_dev: 0.0312


### Sub-objective approach 2

In [11]:
train_sampler = ExpandSubgraph(args.n_ent, args.n_rel,
                               train_graph_homo,train_graph,
                               args=args, use_sub_objectives_b=True)

query_1_hop_stat_b = []
query_2_hop_stat_b = []
query_3_hop_stat_b = []

for _ in range(2):
    loader.shuffle_train()
    train_sampler.updateEdges(loader.train_graph)
    random.shuffle(query_1_hop)
    random.shuffle(query_2_hop)
    random.shuffle(query_3_hop)
    query_1_hop_stat_b.append(calculate_overall_score(query_1_hop[:50], train_sampler, label = "subojective"))
    query_2_hop_stat_b.append(calculate_overall_score(query_2_hop[:50], train_sampler, label = "subojective"))
    query_3_hop_stat_b.append(calculate_overall_score(query_3_hop[:50], train_sampler, label = "subojective"))
    print(f"epoch {_}")

==> removing 1-hop links...
==> done
epoch 0
==> removing 1-hop links...
==> done
epoch 1


In [12]:
from utils import calculate_statistics


stats_1_hop_b = calculate_statistics(query_1_hop_stat_b)
print_statistics(stats_1_hop_b, "1-Hop Query")
stats_2_hop_b = calculate_statistics(query_2_hop_stat_b)
print_statistics(stats_2_hop_b, "2-Hop Query")
stats_3_hop_b = calculate_statistics(query_3_hop_stat_b)
print_statistics(stats_3_hop_b, "3-Hop Query")

1-Hop Query Statistics:
  mean: 0.2926
  std_dev: 0.0399
2-Hop Query Statistics:
  mean: 0.1010
  std_dev: 0.0069
3-Hop Query Statistics:
  mean: 0.0475
  std_dev: 0.0114


### Analyzing the size of retrieved subgraph

In [57]:
avg_size = 0
for i in range(len(all_subgraph["p2s2"])):
    subgraph = all_subgraph["p2s2"][i]["subgraph"]
    # print(subgraph)
    # print(subgraph[:,[0,2]].flatten())
    number_of_nodes = np.unique(subgraph[:,[0,2]].flatten()).shape[0]
    # print(number_of_nodes)
    avg_size += number_of_nodes
    # break
avg_size /= len(all_subgraph["p2s2"])
avg_size

38.31111111111111

### output result

In [ ]:
import json
results = {
    "mine":{
        "1-hop": stats_1_hop,
        "2-hop": stats_2_hop,
        "3-hop": stats_3_hop
    },
    "bfs": {
        "1-hop": stats_1_hop_bfs,
        "2-hop": stats_2_hop_bfs,
        "3-hop": stats_3_hop_bfs
    },
    "subobjective": {
        "1-hop": stats_1_hop_a,
        "2-hop": stats_2_hop_a,
        "3-hop": stats_3_hop_a
    }
}
with open("results/expand_subgraph/expand_subgraph_results1.json", "w") as f:
    json.dump(results, f, indent=4)

NameError: name 'stats_1_hop_bfs' is not defined

#### Evaluate coverage

**Answer Files Structure:**
+ `valid-easy-answers/test-easy-answers.pkl`: defaultdict(set)
  - Each key represents a query
  - Value represents the answers obtained in the training graph (edges in `train.txt`) / valid graph (edges in `train.txt` + `valid.txt`)

+ `valid-hard-answers/test-hard-answers.pkl`: defaultdict(set)  
  - Each key represents a query
  - Value represents the additional answers obtained in the validation graph (edges in `train.txt` + `valid.txt`) / test graph (edges in `train.txt` + `valid.txt` + `test.txt`)